In [1]:
import re
import inflect

from datasets import load_dataset
from torch.utils.data import DataLoader

In [10]:
from allennlp.predictors import Predictor
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")
# predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/fine-grained-ner.2020-06-24.tar.gz")

In [3]:
datasets = load_dataset('experiments/social_bias_frames.py')

Using custom data configuration default
Reusing dataset social_bias_frames (/Users/otakumesi/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/1f541c4093d7d574ebc9638eeef12a0e247e339d90402c294272a414e259996b)


In [29]:
with open('data/masking_words.txt') as f:
    masking_words = [line.strip() for line in f.readlines()]

In [5]:
loader = DataLoader(datasets["train"])

In [ ]:
sentences = []
for i, data in enumerate(loader):
    text = data["post"][0]
    results = predictor.predict(sentence=text)
    words = [results["words"][i] for i, tag in enumerate(results["tags"]) if any([tag.endswith(ent_name) for ent_name in ["PER", "LOC", "MISC"]])]
    sentence = text
    for word in words:
        sentence = re.sub(re.escape(word), "[MASK]", sentence, flags=re.IGNORECASE)
    for word in masking_words:
        sentence = re.sub(rf"\b({word}|{word}'s)\b", "[MASK]", sentence, flags=re.IGNORECASE)
    sentences.append(sentence)

All the pictures of the ex are gone...there's no pictures with the side hoe posted...*slides into his dm's like* http://t.co/XMVtZ0GTPp ['*slides']
All the pictures of the ex are gone...there's no pictures with the side hoe posted...[MASK] into [MASK] dm's like* http://t.co/XMVtZ0GTPp
RT @LOHANTHONY: lana uses the term "bad bitch" in a UV track. i've never been more proud. ['lana']
RT @LOHANTHONY: [MASK] uses the term "bad bitch" in a UV track. i've never been more proud.
RT @LOHANTHONY: lana uses the term "bad bitch" in a UV track. i've never been more proud. ['lana', 'UV']
RT @LOHANTHONY: [MASK] uses the term "bad bitch" in a [MASK] track. i've never been more proud.
RT @LOHANTHONY: lana uses the term "bad bitch" in a UV track. i've never been more proud. ['lana', 'UV']
RT @LOHANTHONY: [MASK] uses the term "bad bitch" in a [MASK] track. i've never been more proud.
RT @UrsTruly_Jessie: These bitches is actors out here ['@UrsTruly_Jessie']
RT [MASK]: These bitches is actors out here
RT

In [ ]:
for sent in sentences:
    print(sent)